In [1]:
def CopyDynamicsData(lakeName):
  from datetime import date
  
  print("LakeName:", lakeName)
  
  # get last date from the loaded dataset
  LastSyncDate = GetLastSyncDate('string_map', 'dynamics')
  
  print("Reading data since", LastSyncDate)  
  print("Load String_Map data")
  PrepareDataBronzeSimpleWithoutInstance (
    lakeName = lakeName,
    entity = BronzeTable.StringMap,
    entityName = 'tmpStringMap'
  )
  
  print("Writing data...")
  
  sourceData = spark.sql("""
    select
          s.value
        , s.attributename
        , s.langid
        , s.objecttypecode
        , s.attributevalue
        , s.stringmapid
        , s.organizationid
        , s.displayorder   
        , current_timestamp() as SilverModifiedUtc
    from tmpStringMap s
  """)  
  
  sourceData.registerTempTable("SourceData")
   
  spark.sql("""
    merge into silver.StringMap as t
    using SourceData as s
      on t.stringmapid = s.stringmapid
    when matched then update set 
        value = s.value
      , attributename = s.attributename
      , langid = s.langid
      , objecttypecode = s.objecttypecode
      , attributevalue = s.attributevalue
      , organizationid = s.organizationid
      , displayorder = s.displayorder
      , SilverModifiedUtc = s.SilverModifiedUtc
    when not matched then 
      insert ( stringmapid, value, attributename, langid, objecttypecode, attributevalue, organizationid, displayorder, SilverModifiedUtc )
      values ( stringmapid, value, attributename, langid, objecttypecode, attributevalue, organizationid, displayorder, SilverModifiedUtc )
  """);

  print("Writing data completed.")
  
  LastSyncDate = date.today()
  print(LastSyncDate)

  if LastSyncDate is not None:
    UpdateLastSyncDate(LastSyncDate, 'string_map', 'dynamics')